<a href="https://colab.research.google.com/github/longbui23/Amazon-Sentiment-Analysis/blob/main/Bigrams_and_Trigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id="1"></a>
<font color="blue" size=+2.5><b>1. Datasets and Libraries</b></font>

## **Import libraries and dataset**

In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/ECON314: Amazon-Sentiment-Analysis

Mounted at /content/drive
/content/drive/My Drive/ECON314: Amazon-Sentiment-Analysis


In [3]:
!pip install langdetect
import nltk

nltk.download('punkt')
nltk.download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=c80dd43c38dff8c4b52ae7bfe58d40f2b9af0483f293e5e8c234190f6ee4d0f1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
import requests
import nltk
import pandas as pd
import string
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect
from nltk import pos_tag
from nltk import RegexpParser
from textblob import TextBlob

In [5]:
df = pd.read_csv("Data/clean_data.csv")
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Review Length
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,374.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,214.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,12.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,76.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,223.0


## **data cleaning**

In [6]:
score = []
for comment in df['Reviews']:
  blob = TextBlob(comment)
  polarity = blob.sentiment.polarity
  score.append(polarity)

In [7]:
df['sentimentScore'] = score

In [8]:
sentimentCategory = []
for eachScore in df['sentimentScore']:
  if eachScore <= -0.7:
    sentimentCategory.append("very negative")
  elif eachScore < -0.7 and eachScore > -0.3:
    sentimentCategory.append("negative")
  elif eachScore < 0 and eachScore >= -0.3:
    sentimentCategory.append("Neutral with Negative Feedback")
  elif eachScore == 0:
    sentimentCategory.append("neutral")
  elif eachScore <= 0.3 and eachScore > 0:
    sentimentCategory.append("Neutral with Positive Feedback")
  elif eachScore < 0.7 and eachScore > 0.3:
     sentimentCategory.append("positive")
  else:
    sentimentCategory.append("very positive")

df['sentiment'] = sentimentCategory

In [9]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Review Length,sentimentScore,sentiment
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,374.0,0.180952,Neutral with Positive Feedback
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,214.0,0.449259,positive
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,12.0,0.650000,positive
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,76.0,0.452500,positive
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,223.0,0.214583,Neutral with Positive Feedback


<a id="1"></a>
<font color="blue" size=+2.5><b>2. Create a function for Word Extraction</b></font>

## **Clean text function**

In [10]:
def cleaned_review(data):

    # Remove numbers
    no_num = data['Reviews'].apply(lambda text: re.sub(r'\b\d+\b', '', text))

    # Remove link
    no_link = no_num.apply(lambda text: re.sub(r'https?://\S+|www\.\S+', '', text))

    # Remove html elements
    no_html = no_link.apply(lambda text: re.sub('<.*?>', '', text))


    # Tokenize words
    tokenized_text = no_html.apply(lambda text: word_tokenize(text))

    # Lowercase the text
    lower_text = [word.lower() for i in tokenized_text for word in i]

    # Remove stopwords
    stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
    stopwords = set(stopwords_list.decode().splitlines())
    nostopword = [word for word in lower_text if word not in stopwords]


    # Remove punctuation
    punc = set(string.punctuation)
    no_pun = [element for element in nostopword if element not in punc]


    # Lematize words
    lem = WordNetLemmatizer()
    lem_word = [lem.lemmatize(word) for word in no_pun]

    #remove customized stopwords:
    unwantedWord = ["sim", "card", "n't", "samsung", "galaxy", "'s", "unlocked", "--","``","''", "received", "purchases", "bought", "purchased", "unlocked","refurbished","unlock" ]
    finalList = [word for word in lem_word if word not in unwantedWord]


    return finalList

## **Bigram Function**

In [11]:
def bigrams_phrases(word_list):

    # make biagrams:
    output = nltk.bigrams(word_list)

    # count biagrams:
    pair_frequency = {}
    for pair in output:
        if pair in pair_frequency:
            pair_frequency[pair] += 1
        else:
            pair_frequency[pair] = 1

    # sort biagrams based on frequency
    sorted_pairs = sorted(pair_frequency.items(), key=lambda x: x[1], reverse=True)

    # Create a data frame:
    pairs_words = pd.DataFrame(sorted_pairs, columns=['Pair', 'Frequency'])

    # Get the top 15 popular pairs:
    popular_pairs = pairs_words.head(30)
    return popular_pairs

## **trigrams function**

In [41]:
def trigram_phrases(word_list):

    # make biagrams:
    output = nltk.trigrams(word_list)

    # count biagrams:
    pair_frequency = {}
    for pair in output:
        if pair in pair_frequency:
            pair_frequency[pair] += 1
        else:
            pair_frequency[pair] = 1

    # sort biagrams based on frequency
    sorted_pairs = sorted(pair_frequency.items(), key=lambda x: x[1], reverse=True)

    # Create a data frame:
    pairs_words = pd.DataFrame(sorted_pairs, columns=['Pair', 'Frequency'])

    # Get the top 30 popular pairs:
    popular_pairs = pairs_words.head(40)
    #print(popular_pairs)
    return popular_pairs

# Brand Battle

## Samsung

In [ ]:
dfSamsung = df[df['Brand Name'] == "Samsung"]
dfSamsung.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Review Length,sentimentScore,sentiment
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,374.0,0.180952,Neutral with Positive Feedback
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,214.0,0.449259,positive
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,12.0,0.650000,positive
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,76.0,0.452500,positive
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,223.0,0.214583,Neutral with Positive Feedback


### *negative reviews key words*

In [ ]:
samsungNegatve = dfSamsung[dfSamsung['sentimentScore'] < 0 ]
samsungNegList = cleaned_review(samsungNegatve)
print(bigrams_phrases(samsungNegList))
print(trigram_phrases(samsungNegList))

                        Pair  Frequency
0             (phone, phone)        460
1              (phone, work)        448
2               (buy, phone)        323
3              (cell, phone)        253
4             (phone, month)        224
5         (stopped, working)        213
6            (battery, life)        198
7             (waste, money)        190
8      (disappointed, phone)        183
9           (problem, phone)        131
10  (international, version)        124
11        ('m, disappointed)        121
12           (phone, screen)        118
13           (return, phone)        116
14              (phone, bad)        115
15            (month, phone)        115
16       (customer, service)        114
17            (worst, phone)        113
18            (money, phone)        107
19           (phone, worked)        107
20           (touch, screen)        105
21             (work, phone)        105
22             (phone, slow)        103
23             (time, phone)         97


### *Positive reviews key words*

In [ ]:
samsungPositive = dfSamsung[dfSamsung['sentimentScore'] >= 0.3 ]
samsungPosList = cleaned_review(samsungPositive)
print(bigrams_phrases(samsungPosList))
print(trigram_phrases(samsungPosList))

                    Pair  Frequency
0         (great, phone)       3699
1          (love, phone)       2403
2          (work, great)       1980
3         (phone, great)       1952
4          (good, phone)       1916
5          (phone, work)       1658
6          (phone, good)       1391
7         (phone, phone)        904
8          (nice, phone)        899
9   (excellent, product)        852
10    (excellent, phone)        848
11         (phone, love)        793
12     (work, perfectly)        788
13       (battery, life)        784
14       (good, product)        713
15      (great, product)        680
16         (cell, phone)        669
17    (phone, excellent)        625
18          (good, good)        587
19        (great, price)        535
20      (phone, perfect)        515
21      (awesome, phone)        506
22          (work, fine)        462
23          (love, love)        457
24        (smart, phone)        448
25         (good, great)        443
26        (happy, phone)    

## Apple

In [ ]:
dfApple = df[df['Brand Name'] == "Apple"]
dfApple.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Review Length,sentimentScore,sentiment
3041,Apple - Iphone 5c A1532 Verizon 16 GB Cell Pho...,Apple,33.0,5,Was a gift for my husband and he loves it. Rep...,5.0,260.0,0.068074,Neutral with Positive Feedback
3042,Apple - Iphone 5c A1532 Verizon 16 GB Cell Pho...,Apple,33.0,5,"Great price, extremely fast shipping works as ...",1.0,57.0,0.525000,positive
3043,Apple - Iphone 5c A1532 Verizon 16 GB Cell Pho...,Apple,33.0,4,Moving from Android to Apple...my kids have th...,1.0,337.0,0.214394,Neutral with Positive Feedback
3044,Apple - Iphone 5c A1532 Verizon 16 GB Cell Pho...,Apple,33.0,4,It is compatible with Verizon. Not pink but PE...,0.0,174.0,0.162500,Neutral with Positive Feedback
3045,Apple - Iphone 5c A1532 Verizon 16 GB Cell Pho...,Apple,33.0,5,I love this phone! Thank you so much.,1.0,37.0,0.412500,positive


### *negative reviews key words*

In [ ]:
appleNegatve = dfApple[dfApple['sentimentScore'] < 0 ]
appleNegList = cleaned_review(appleNegatve)
print(bigrams_phrases(appleNegList))
print(trigram_phrases(appleNegList))

                     Pair  Frequency
0           (phone, work)        405
1          (phone, phone)        334
2          (apple, store)        263
3            (buy, phone)        243
4      (stopped, working)        198
5          (waste, money)        195
6         (battery, life)        189
7           (waste, time)        169
8          (hold, charge)        164
9          (phone, month)        123
10         (button, work)        122
11        (buying, phone)        114
12  (disappointed, phone)        113
13         (wrong, phone)        111
14           (..., phone)        109
15      (purchase, phone)        108
16       (phone, battery)        107
17         (money, phone)        105
18        (phone, locked)        105
19       (problem, phone)        104
20        (return, phone)        104
21          (work, phone)        103
22       (verizon, phone)         99
23        (touch, screen)         99
24            (iphone, 5)         98
25          (time, money)         98
2

### *positive reviews key words*

In [ ]:
applePositive = dfSamsung[dfSamsung['sentimentScore'] >= 0.3 ]
applePosList = cleaned_review(applePositive)
print(bigrams_phrases(applePosList))
print(trigram_phrases(applePosList))

                    Pair  Frequency
0         (great, phone)       3699
1          (love, phone)       2403
2          (work, great)       1980
3         (phone, great)       1952
4          (good, phone)       1916
5          (phone, work)       1658
6          (phone, good)       1391
7         (phone, phone)        904
8          (nice, phone)        899
9   (excellent, product)        852
10    (excellent, phone)        848
11         (phone, love)        793
12     (work, perfectly)        788
13       (battery, life)        784
14       (good, product)        713
15      (great, product)        680
16         (cell, phone)        669
17    (phone, excellent)        625
18          (good, good)        587
19        (great, price)        535
20      (phone, perfect)        515
21      (awesome, phone)        506
22          (work, fine)        462
23          (love, love)        457
24        (smart, phone)        448
25         (good, great)        443
26        (happy, phone)    

## Motorola


In [ ]:
dfMotorola = df[df['Brand Name'] == "Motorola"]
dfMotorola.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Review Length,sentimentScore,sentiment
81118,AT&T Motorola RAZR V3 No Contract Quad Band GS...,Motorola,26.99,1,Not good,0.0,8.0,-0.35,very positive
81119,AT&T Motorola RAZR V3 No Contract Quad Band GS...,Motorola,26.99,1,junk did not work got a refund,0.0,30.0,0.00,neutral
81120,AT&T Motorola RAZR V3 No Contract Quad Band GS...,Motorola,26.99,3,"I love the looks of it but,it has no S.D.slot....",0.0,127.0,0.25,Neutral with Positive Feedback
81121,AT&T Motorola RAZR V3 No Contract Quad Band GS...,Motorola,26.99,4,Got phone it had no scratches but can't find w...,0.0,191.0,0.30,Neutral with Positive Feedback
81122,AT&T Motorola RAZR V3 No Contract Quad Band GS...,Motorola,26.99,4,Works great,0.0,11.0,0.80,very positive


### *negative reviews key words*

In [ ]:
MotorolaNegatve = dfMotorola[dfMotorola['sentimentScore'] < 0 ]
motorolaList = cleaned_review(MotorolaNegatve)
print(bigrams_phrases(motorolaList))
print(trigram_phrases(motorolaList))

                     Pair  Frequency
0           (phone, work)         94
1          (phone, phone)         92
2         (battery, life)         91
3           (cell, phone)         68
4            (buy, phone)         64
5          (waste, money)         52
6          (phone, month)         42
7   (disappointed, phone)         42
8          (worst, phone)         41
9           (phone, slow)         39
10         (month, phone)         35
11        (charge, phone)         33
12    (customer, service)         32
13       (phone, battery)         32
14     (stopped, working)         32
15     (recommend, phone)         31
16        (phone, worked)         31
17       (verizon, phone)         29
18          (piece, crap)         29
19         (hold, charge)         28
20           (phone, day)         28
21          (phone, week)         28
22      (motorola, phone)         28
23        (return, phone)         28
24          (phone, call)         27
25       (ordered, phone)         27
2

### *positive reviews key words*




In [ ]:
motorolaPositive = dfMotorola[dfMotorola['sentimentScore'] >= 0.3 ]
motorolaPosList = cleaned_review(motorolaPositive)
print(bigrams_phrases(motorolaPosList))
print(trigram_phrases(motorolaPosList))

                    Pair  Frequency
0         (great, phone)        858
1         (phone, great)        472
2          (good, phone)        395
3          (work, great)        362
4          (phone, good)        326
5          (love, phone)        305
6        (battery, life)        277
7          (phone, work)        258
8         (phone, price)        243
9         (phone, phone)        209
10         (cell, phone)        186
11        (smart, phone)        178
12         (nice, phone)        173
13    (excellent, phone)        170
14        (great, price)        153
15         (phone, love)        141
16    (phone, excellent)        127
17      (awesome, phone)        123
18         (good, price)        118
19        (price, great)        100
20        (happy, phone)         98
21     (work, perfectly)         94
22         (price, good)         93
23      (great, product)         92
24         (good, great)         91
25          (work, fine)         88
26      (phone, perfect)    

**II. WORD PHRASES BY PRICE**

In [61]:
import numpy as np

q1 = np.quantile(df['Price'], 0.25)
q2 = np.quantile(df['Price'], 0.5)
q3 = np.quantile(df['Price'], 0.7)
q4 = np.quantile(df['Price'], 0.9)

print(q1,q2,q3,q4)

84.99 149.99 240.0 514.98


In [66]:
import numpy as np
import pandas as pd

#quant val
quantile_values = [0,0.25,0.5,0.7,0.9, 1]

#negative sentiment
df_neg = df[df['sentimentScore'] < 0]

# df for bigrams and trigrams
df_bigrams = pd.DataFrame()
df_trigrams = pd.DataFrame()

#segment quantile values
df_neg['Price_Quantile'] = pd.qcut(df_neg['Price'], quantile_values, labels=['Q1', 'Q2', 'Q3', 'Q4','Q5'])

#length of each
for qlab in ['Q1', 'Q2', 'Q3', 'Q4','Q5'] :
  df_quantile = df_neg[df_neg['Price_Quantile'] == qlab]
  print(len(df_quantile))

12365
12336
9882
9965
4853


<ipython-input-66-77a063d47caa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['Price_Quantile'] = pd.qcut(df_neg['Price'], quantile_values, labels=['Q1', 'Q2', 'Q3', 'Q4','Q5'])


In [67]:
# Loop through each quantile
for qlab in ['Q1', 'Q2', 'Q3', 'Q4','Q5'] :
  df_quantile = df_neg[df_neg['Price_Quantile'] == qlab]

  #clean cmts
  segment_price_list = cleaned_review(df_quantile)

  #bigrams
  bigram_q = bigrams_phrases(segment_price_list)
  df_bigrams[['pair'+qlab,'freqency'+qlab]]= bigram_q[['Pair','Frequency']]

  #triagrams
  trigram_q = trigram_phrases(segment_price_list)
  df_trigrams[['pair'+qlab,'frequency'+qlab]]= trigram_q[['Pair','Frequency']]

In [68]:
df_bigrams

,pairQ1,freqencyQ1,pairQ2,freqencyQ2,pairQ3,freqencyQ3,pairQ4,freqencyQ4,pairQ5,freqencyQ5
0,"(waste, money)",619,"(phone, work)",576,"(phone, phone)",508,"(phone, work)",484,"(phone, phone)",185
1,"(phone, phone)",572,"(phone, phone)",564,"(phone, work)",458,"(phone, phone)",433,"(phone, work)",177
2,"(phone, work)",478,"(waste, money)",475,"(buy, phone)",331,"(buy, phone)",340,"(buy, phone)",136
3,"(buy, phone)",388,"(buy, phone)",474,"(waste, money)",312,"(stopped, working)",298,"(battery, life)",131
4,"(battery, life)",328,"(battery, life)",441,"(battery, life)",305,"(battery, life)",280,"(phone, month)",124
5,"(cell, phone)",317,"(stopped, working)",307,"(phone, month)",294,"(cell, phone)",214,"(customer, service)",104
6,"(stopped, working)",249,"(cell, phone)",288,"(stopped, working)",294,"(phone, month)",212,"(stopped, working)",102
7,"(touch, screen)",238,"(phone, month)",283,"(cell, phone)",238,"(disappointed, phone)",187,"(disappointed, phone)",87
8,"(phone, month)",206,"(worst, phone)",254,"(customer, service)",199,"(waste, money)",178,"(waste, money)",82
9,"(worst, phone)",202,"(phone, battery)",201,"(disappointed, phone)",170,"(touch, screen)",152,"(month, phone)",75


In [69]:
df_trigrams

,pairQ1,frequencyQ1,pairQ2,frequencyQ2,pairQ3,frequencyQ3,pairQ4,frequencyQ4,pairQ5,frequencyQ5
0,"(waste, money, phone)",66,"(waste, money, phone)",78,"(phone, stopped, working)",73,"(phone, stopped, working)",70,"(stopped, working, month)",21
1,"(phone, waste, money)",59,"(phone, stopped, working)",61,"(waste, money, phone)",72,"(stopped, working, month)",54,"(quality, jet, black)",20
2,"(phone, mp, camera)",56,"(waste, time, money)",59,"(stopped, working, month)",53,"(phone, month, ago)",44,"(phone, month, ago)",19
3,"(international, version, warranty)",56,"(stopped, working, month)",41,"(phone, month, ago)",41,"(waste, time, money)",40,"(poor, battery, life)",19
4,"(version, warranty, black)",55,"(battery, life, suck)",39,"(waste, time, money)",40,"(phone, buy, phone)",27,"(waste, time, money)",18
5,"(mp, camera, bluetooth)",54,"(phone, buy, phone)",39,"(phone, battery, life)",37,"(battery, hold, charge)",26,"(...., waste, time)",18
6,"(camera, bluetooth, wi-fi)",54,"(phone, worst, phone)",38,"(phone, hold, charge)",34,"(phone, worst, phone)",26,"(phone, stopped, working)",17
7,"(bluetooth, wi-fi, international)",54,"(phone, hold, charge)",34,"(total, waste, money)",33,"(camera, stopped, working)",26,"(phone, box, opened)",17
8,"(wi-fi, international, version)",54,"(total, waste, money)",34,"(battery, hold, charge)",30,"(stopped, working, week)",25,"(box, opened, phone)",17
9,"(touch, screen, work)",45,"(money, buy, phone)",33,"(battery, life, terrible)",28,"(hard, earned, money)",24,"(item, iphone, box)",16


**POSITIVE CMTS**

In [72]:
import numpy as np

q1 = np.quantile(df['Price'], 0.25)
q2 = np.quantile(df['Price'], 0.5)
q3 = np.quantile(df['Price'], 0.7)
q4 = np.quantile(df['Price'], 0.9)

print(q1,q2,q3,q4)

84.99 149.99 240.0 514.98


In [70]:
#quant val
quantile_values = [0,0.25,0.5,0.7,0.9, 1]

#negative sentiment
df_neg = df[df['sentimentScore'] >= 0.3]

# df for bigrams and trigrams
df_bigrams = pd.DataFrame()
df_trigrams = pd.DataFrame()

#segment quantile values
df_neg['Price_Quantile'] = pd.qcut(df_neg['Price'], quantile_values, labels=['Q1', 'Q2', 'Q3', 'Q4','Q5'])

#length of each
for qlab in ['Q1', 'Q2', 'Q3', 'Q4','Q5'] :
  df_quantile = df_neg[df_neg['Price_Quantile'] == qlab]
  print(len(df_quantile))

47005
46994
37796
37408
18786


<ipython-input-70-a61378d0f97c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['Price_Quantile'] = pd.qcut(df_neg['Price'], quantile_values, labels=['Q1', 'Q2', 'Q3', 'Q4','Q5'])


In [71]:
# Loop through each quantile
for qlab in ['Q1', 'Q2', 'Q3', 'Q4','Q5'] :
  df_quantile = df_neg[df_neg['Price_Quantile'] == qlab]

  #clean cmts
  segment_price_list = cleaned_review(df_quantile)

  #bigrams
  bigram_q = bigrams_phrases(segment_price_list)
  df_bigrams[['pair'+qlab,'freqency'+qlab]]= bigram_q[['Pair','Frequency']]

  #triagrams
  trigram_q = trigram_phrases(segment_price_list)
  df_trigrams[['pair'+qlab,'frequency'+qlab]]= trigram_q[['Pair','Frequency']]

In [73]:
df_bigrams

,pairQ1,freqencyQ1,pairQ2,freqencyQ2,pairQ3,freqencyQ3,pairQ4,freqencyQ4,pairQ5,freqencyQ5
0,"(great, phone)",3363,"(great, phone)",4603,"(great, phone)",3792,"(great, phone)",3527,"(great, phone)",1312
1,"(work, great)",2658,"(phone, great)",2704,"(love, phone)",2180,"(love, phone)",2067,"(love, phone)",987
2,"(good, phone)",2577,"(work, great)",2621,"(phone, great)",2118,"(phone, great)",1849,"(phone, great)",809
3,"(phone, great)",1986,"(good, phone)",2572,"(work, great)",1966,"(work, great)",1571,"(work, great)",776
4,"(phone, good)",1956,"(love, phone)",2568,"(good, phone)",1797,"(good, phone)",1544,"(good, phone)",702
5,"(phone, work)",1856,"(phone, work)",1895,"(phone, work)",1561,"(phone, work)",1225,"(phone, work)",556
6,"(love, phone)",1775,"(phone, good)",1887,"(phone, good)",1411,"(phone, good)",1198,"(phone, good)",502
7,"(phone, phone)",1065,"(battery, life)",1421,"(battery, life)",993,"(battery, life)",1035,"(battery, life)",405
8,"(nice, phone)",1040,"(nice, phone)",1253,"(phone, phone)",900,"(phone, phone)",853,"(work, perfectly)",373
9,"(good, good)",1029,"(phone, price)",1193,"(nice, phone)",899,"(work, perfectly)",788,"(nice, phone)",336


In [75]:
df_trigrams

,pairQ1,frequencyQ1,pairQ2,frequencyQ2,pairQ3,frequencyQ3,pairQ4,frequencyQ4,pairQ5,frequencyQ5
0,"(phone, work, great)",648,"(phone, work, great)",752,"(phone, work, great)",574,"(great, phone, great)",421,"(phone, work, great)",194
1,"(phone, great, phone)",326,"(great, phone, great)",472,"(great, phone, great)",432,"(phone, work, great)",394,"(phone, great, phone)",130
2,"(great, phone, great)",320,"(phone, great, phone)",469,"(phone, great, phone)",383,"(phone, great, phone)",362,"(great, phone, great)",112
3,"(phone, good, phone)",282,"(great, phone, price)",395,"(phone, great, price)",279,"(phone, love, phone)",207,"(great, phone, work)",86
4,"(good, phone, good)",250,"(phone, great, price)",375,"(great, phone, price)",219,"(good, great, phone)",201,"(phone, work, perfectly)",77
5,"(great, phone, price)",246,"(phone, good, phone)",293,"(great, phone, work)",215,"(phone, great, price)",198,"(great, phone, love)",77
6,"(phone, great, price)",222,"(phone, love, phone)",285,"(great, phone, good)",198,"(great, phone, good)",197,"(phone, good, phone)",76
7,"(great, phone, good)",209,"(great, phone, good)",249,"(phone, good, phone)",194,"(phone, work, perfectly)",180,"(work, great, love)",75
8,"(good, phone, price)",205,"(good, phone, price)",219,"(phone, love, phone)",190,"(great, phone, love)",176,"(love, love, phone)",74
9,"(work, great, good)",197,"(good, great, phone)",205,"(good, great, phone)",181,"(phone, perfect, condition)",161,"(work, great, good)",73
